In [58]:
import hashlib
import json
import os
import pandas as pd
import re

In [88]:
exp_dir = '../experiment_data/pilot_june9/'
csv_name = 'mturk.csv'
log_dir = 'www/log/'
feedback_dir = 'www/feedback'

surveycode_hash = 'thowpbr_pilot0'  # pilot_jun9

max_nav_len = 300
max_dialog_len = 50

In [104]:
# Read in the CSV from Turk and connect each player to their respective logfile to rebuild games.
players = pd.DataFrame(columns=["uid", "pid", "role", "rating", "feedback"])
games = pd.DataFrame(columns=["navigator", "oracle", "navigator_quality", "oracle_quality",
                              "house", "target", "start_pano", "end_panos",
                              "success", "nav_trajectory", "dialog_history", "stop_history",
                              "num_nav_steps", "num_dialog_turns", "num_stop_attempts"])
mturk = pd.read_csv(os.path.join(exp_dir, csv_name))
for idx in mturk.index:
    # Approve surveycode.
    surveycode = mturk['Answer.surveycode'][idx]
    wid = mturk['WorkerId'][idx]
    if '_' not in surveycode or surveycode.count('_') != 1:
        print('WARNING: worker %s has invalid surveycode %s' % (wid, surveycode))
        continue
    uid, hashcode = mturk['Answer.surveycode'][idx].split('_')
    hash_str = 'phm_salted_hash%s%s' % (uid, surveycode_hash)
    correcthash = hashlib.sha1(hash_str.encode('utf-8')).hexdigest()[:13]
    if correcthash != hashcode:
        print('WARNING: worker %s has wrong hash %s does not match target %s' % (wid, hashcode, correcthash))
        continue
        
    # Find the feedback file for this user.
    uid_feedback_fn = os.path.join(exp_dir, feedback_dir, '%s.json' % uid)
    if not os.path.isfile(uid_feedback_fn):
        print('WARNING: worker %s with uid %s has no feedback file' % (wid, uid))
        continue
    with open(os.path.join(exp_dir, feedback_dir, '%s.json' % uid)) as f:
        feedback = json.load(f)
    role = 'navigator' if feedback['navigator'] == uid else 'oracle'
    pid_role = 'navigator' if role == 'oracle' else 'oracle'
    pid = feedback['navigator'] if role == 'oracle' else feedback['oracle']
    pid_rating = int(feedback['rating']) if feedback['rating'] is not None else None
    str_feedback = feedback['free_form_feedback']
    
    # Add self and partner to players dataframe.
    for add_uid in [uid, pid]:
        if add_uid not in players['uid']:
            players = players.append({
                        "uid": add_uid,
                        "pid": None,
                        "role": None,
                        "rating": None,
                        "feedback": None,
                    }, ignore_index=True)
    uid_idx = players.index[players['uid'] == uid].tolist()[0]
    players['pid'][uid_idx] = pid
    players['role'][uid_idx] = role
    players['feedback'][uid_idx] = str_feedback
    pid_idx = players.index[players['uid'] == pid].tolist()[0]
    players['rating'][pid_idx] = pid_rating
    
    # Find the logfile and partner id for this user.
    for _, _, fns in os.walk(os.path.join(exp_dir, log_dir)):
        for fn in fns:
            if ('.' in fn and fn.split('.')[-1] == 'log' and
                '_' in fn and uid in fn.split('.')[0].split('_') and pid in fn.split('.')[0].split('_')):
    
                # Already have game information for this user.
                if pid in games['navigator']:
                    games['oracle'][games['navigator'].index(pid)] = uid
                    games['navigator_quality'][games['navigator'].index(pid)] = pid_rating
                elif pid in games['oracle']:
                    games['navigator'][games['oracle'].index(pid)] = uid
                    games['oracle_quality'][games['oracle'].index(pid)] = pid_rating

                # Need to create game information for this unseen pair.
                else:
                    navigator = oracle = navigator_quality = oracle_quality = None
                    house = target = start_pano = end_panos = None
                    success = False
                    nav_trajectory = [None] * max_nav_len
                    dialog_history = [None] * max_dialog_len
                    stop_history = [None] * max_dialog_len
                    num_nav_steps = num_dialog_turns = num_stop_attempts = 0
                    curr_pano = None
                    curr_turn = "navigator"
                    with open(os.path.join(exp_dir, log_dir, fn)) as f:
                        logfile_lines = f.readlines()
                        for line in logfile_lines:
                            time, sender, data_str = line.strip().split('\t')
                            data_str = re.sub(r'(?!(([^"]*"){2})*[^"]*$)\'', "\\'", data_str)
                            data_str = data_str.replace("'", '"')
                            line_data = json.loads(data_str)
                            
                            if line_data['action'] == 'set_house':
                                house = line_data['value']
                            elif line_data['action'] == 'set_target_obj':
                                target = line_data['value']
                            elif line_data['action'] == 'set_start_pano':
                                start_pano = line_data['value']
                                curr_pano = start_pano
                                nav_trajectory[num_nav_steps] = curr_pano
                                num_nav_steps += 1
                            elif line_data['action'] == 'set_end_panos':
                                start_pano = line_data['value']
                            elif line_data['action'] == 'enable_nav':
                                navigator = uid
                                oracle = pid
                                oracle_quality = pid_rating
                            elif line_data['action'] == 'show_gold_view':
                                oracle = uid
                                navigator = pid
                                navigator_quality = pid_rating
                            elif line_data['action'] == 'nav' and line_data['message']['img_id'] != curr_pano:
                                curr_pano = line_data['message']['img_id']
                                nav_trajectory[num_nav_steps] = curr_pano
                                num_nav_steps += 1
                            elif line_data['action'] == 'chat':
                                if curr_turn == role:
                                    dialog_history[num_dialog_turns] = {"uid": uid,
                                                                        "role": role,
                                                                        "nav_idx": num_nav_steps - 1,
                                                                        "message": line_data['message']}
                                else:
                                    dialog_history[num_dialog_turns] = {"uid": pid,
                                                                        "role": pid_role,
                                                                        "nav_idx": num_nav_steps - 1,
                                                                        "message": line_data['message']}
                                num_dialog_turns += 1
                                curr_turn = "navigator" if curr_turn == "oracle" else "oracle"
                            elif line_data['action'] == 'guess_stop':
                                stop_history[num_stop_attempts] = num_nav_steps
                                num_stop_attempts += 1
                            elif line_data['action'] == 'set_aux' and 'Congrats, ' in line_data['message']:
                                success = True
                                
                        games = games.append({
                            "navigator": navigator,
                            "oracle": oracle,
                            "navigator_quality": navigator_quality,
                            "oracle_quality": oracle_quality,
                            "house": house,
                            "target": target,
                            "start_pano": start_pano,
                            "end_panos": end_panos,
                            "success": success,
                            "nav_trajectory": nav_trajectory,
                            "dialog_history": dialog_history,
                            "stop_history": stop_history,
                            "num_nav_steps": num_nav_steps,
                            "num_dialog_turns": num_dialog_turns,
                            "num_stop_attempts": num_stop_attempts,
                        }, ignore_index=True)
print("Got %d players over %d games" % (len(players), len(games)))

Got 36 players over 8 games


In [105]:
# Inspect open feedback.
for idx in players.index:
    if players['feedback'][idx] is not None and len(players['feedback'][idx]) > 0:
        print("%s: %s" % (players['uid'][idx], players['feedback'][idx].strip()))

5cfd5e36be024: They wandered around for a bit before asking, but otherwise asked pretty well.
5cfd5da427022: They never said anything, so I couldn't do anything.
5cfd5daab01cc: I had a strong idea of where the room was based on their comment.
5cfd8f14db630: Too bad I couldn't get a partner, this seems like it really would have been fun.
5cfd81afe391e: no one sadly connected
5cfd6b84b33ac: It was easy but my partner and I understood eachother so maybe that's not always the case
5cfd6bd174a0b: this was fun
5cfd8ccba8114: i tried to reload many times but never could get it to work with a partner
5cfd729b21a0e: He took up almost all of my time! Yeesh, not a good partner.
5cfd72052692c: This was ridiculous.  That wasn't a bathtub it was a shower.  And this hit was grossly underpaid.  Thanks for wasting my time.
5cfd7ba63781c: Thank you.
5cfd7a5d7a420: hey I can wait a little longer.


In [106]:
# Inspect dialogs.
for idx in games.index:
    print("house %s, target %s, dialog len %d, nav len %d, success: " %
          (games['house'][idx], games['target'][idx], games['num_dialog_turns'][idx], games['num_nav_steps'][idx]) +
          str(games['success'][idx]))
    print("navigator quality: %d, oracle quality: %d" % (games['navigator_quality'][idx], games['oracle_quality'][idx]))
    for tidx in range(games['num_dialog_turns'][idx]):
        turn = games['dialog_history'][idx][tidx]
        print("\t[%d] %s(%s):\t%s" % (turn['nav_idx'], turn['role'], turn['uid'], turn['message']))
    print()

house wc2JMjhGNzB, target bathtub, dialog len 10, nav len 37, success: True
navigator quality: 5, oracle quality: 5
	[7] navigator(5cfd5de1c26d1):	Should I go to the right or to the left through the door ahead?
	[7] oracle(5cfd5e36be024):	The right
	[9] navigator(5cfd5de1c26d1):	Should I go to the left or to the right?
	[9] oracle(5cfd5e36be024):	I think you can only go left
	[22] navigator(5cfd5de1c26d1):	Which direction should I turn next?
	[22] oracle(5cfd5e36be024):	Right
	[25] navigator(5cfd5de1c26d1):	Now where should I go?
	[25] oracle(5cfd5e36be024):	Go straight until you run into the paintings at the end of the hall.
	[30] navigator(5cfd5de1c26d1):	Okay, should I turn left or right this time?
	[30] oracle(5cfd5e36be024):	Go left and keep going that way until you get to the bedroom. It"s in the adjoined room to the bedroom.

house 17DRP5sb8fy, target toiletpaper, dialog len 0, nav len 10, success: True
navigator quality: 1, oracle quality: 1

house 17DRP5sb8fy, target sink, dia